In [20]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [3]:
warnings.filterwarnings("ignore")
np.random.seed(40)

In [4]:
# Read the wine-quality csv file from the URL
csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
print(f"Number of rows in data {len(data)}")

Number of rows in data 1599


In [6]:
data['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [7]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [11]:
print(f"Number of rows in train {len(train)}")
print(f"Number of rows in test {len(test)}")

Number of rows in train 1199
Number of rows in test 400


In [12]:
# The predicted column is "quality" which is a scalar from [3, 8]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [13]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1316,5.4,0.74,0.00,1.2,0.041,16.0,46.0,0.99258,4.01,0.59,12.5
1507,7.5,0.38,0.57,2.3,0.106,5.0,12.0,0.99605,3.36,0.55,11.4
849,6.4,0.63,0.21,1.6,0.080,12.0,32.0,0.99689,3.58,0.66,9.8
82,7.4,0.50,0.47,2.0,0.086,21.0,73.0,0.99700,3.36,0.57,9.1
644,9.9,0.54,0.45,2.3,0.071,16.0,40.0,0.99910,3.39,0.62,9.4


In [14]:
train_y.head()

,quality
1316,6
1507,6
849,5
82,5
644,5


In [24]:
def eval_metrics(actual, pred):
    f1 = f1_score(actual, pred, average="macro")
    precision = precision_score(actual, pred, average="macro")
    recall = recall_score(actual, pred, average="macro")
    return f1, precision, recall

In [33]:
# model hyper-parameters
l1_ratio = 0.5
max_iter = 100

In [34]:
with mlflow.start_run():
    # Execute ElasticNet
    lr = LogisticRegression(max_iter=max_iter, l1_ratio=l1_ratio, random_state=42, penalty="elasticnet", multi_class="multinomial", solver="saga")
    lr.fit(train_x, train_y)

    # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (f1, precision, recall) = eval_metrics(test_y, predicted_qualities)

    # Print out metrics
    print("Elasticnet model (max_iter=%f, l1_ratio=%.2f):" % (max_iter, l1_ratio))
    print("F1: %s" % f1)
    print("Precision: %s" % precision)
    print("Recall: %s" % recall)

    # Log parameter, metrics, and model to MLflow
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("F1", f1)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (max_iter=100000.000000, l1_ratio=0.40):
F1: 0.22679888575015694
Precision: 0.24324637071323377
Recall: 0.24203364950316167
